# Database Management
Daniel Bultrini

## Why would we want this?

As researchers we handle a lot of data. There is almost no escape unless one escapes to pure mathematics and mathematical physics, perhaps. But even when working on mostly theoretical research, it is almost certain that at some point you will be faced with the prospect of running some numerical simulations, or even generating plots from analytical functions with some parameters. With experiments, it goes without saying that the whole laboratory is built to gather and process data. 

Database management is not strictly necessary to complete even ambitious projects requiring gigabytes of data, with enough patience and coping strategies. Yet having access to tools and techniques that make trawling through experimental and numerical data in a consistent way can help in two major ways. 

### open data
First, as time goes by, more and more journals greatly appreciate open access to both the data and code used to generate the results shown in the paper. Of course, this is not always possible depending on the nature of the research, but even then, you might be asked to turn your code or data over to a reviewer. Having a reliable and easy to share way to access this data is key to make it usable far in the future. 

Second, and more importantly, having your data well organized and easily accessible digitally allows you to explore and plot your dataset in ways that can uncover new relationships and interesting conclusions from work, even if your aims were completely different. Furthermore, typing everything in the style we will explore in this workshop makes it extremely easy to apply powerful techniques from simpler machine learning/regression to deep neural networks to your results, which may or may not be helpful.

## Commonly used tools

### SQL

In big data, SQL databases are king, as they allow for easily distributed storage, incremental addition and "simple" retrievals of arbitrary subsets of the data. Using such a system is simple from python, and allows you to have all of your results in one large, queryable setup. Even if your data gathering changes, new entries don't necessarily have to conform to the old, and the old can be updated to be compatible with the new.

This can happen if you, for example, begin to be able to measure new properties thanks to a new sensor or a fancy new output statement in your code. We will not focus on SQL excessively, but a simple example will be given.

### R

This language comes with a set of tools that are geared towards analysis and is loved by statisticians everywhere, and used by data scientists on a similar level to python.


### Python and Pandas

Python is one of the most commonly used open source languages in multiple branches of science outside of performance critical codes. Yet, even in such cases, python can  integrate with a huge host of languages and programs, indeed it is possible to call functions and other executables to perform any given operation or simulation via a *wrapper* function, or an explicit call to the program (less orthodox, but sometimes required). Prebuilt wrappers or standard recipes exist to all major languages, from C to Fortran to Julia. 

Commercial programs such as MATLAB and Labview even provide libraries that makes sharing data a breeze. Thorlab cameras and sensors are no exception. In the worst case, one can write from one program or function into an intermediary file which is then loaded by python - this is fine for most work, but requires direct involvement.

Pandas offers a fairly simple way to deal with organizing, processing and storing data. It can interface with everything described above and more, depending on your needs. It also comes with its own powerful set of visualization tools, as well as other software written around it that can take some of the pain from statistical analysis and plotting of your work.

# Database hygiene

We are fortunate, in running our own experiments (of whatever type they may be) that we can start structuring our data gathering correctly from the beginning. This means that we don't need to necessarily change and alter what may be very messy databases to be suitable for quick processing. That being said, as experiments change, it will be necessary to alter our structure as time goes by, so 'data cleaning' and recasting will doubtlessly be a part of the process. I hope to show you the tools to be able to do this programmatically throughout this workshop. 

A well structured dataset is tool agnostic, which is to say that the resulting datasets are easy to transfer from program to program, irrespective of what you will choose to use in your own work.

Let us think of two simple ways you might save data:

**Example 1**

| samp | exp_1 | exp_2 | --- |
| ---- | ----- | ----- | --- |
| A    | 10    | -     |     |
| B    | 15    | 6     |     |


**Example 2**

| exp | samp_A | samp_B | 
| --- | ------ | ------ |
| 1   | 10     | 15     |
| 2   | -      | 6      |
| --- | ---    | ---    |

Both of these are adequate to some extent, but imagine that you want to add some extra information, like the date (or experimental parameters).

**Example 1a**

| sample | exp_1 | exp_2 | date    |
| ------ | ----- | ----- | ------- |
| A      | 10    | -     | 11/3/21 |
| B      | 15    | 6     | 12/3/21 |

We already get a problem, that a date in this case, can only be linked to a single sample and not an experiment. Of course this can be somewhat remedied by making the date parameter somewhat complicated as such:


**Example 1b**

| samp | exp_1 | exp_2 | date_exp1 | date_exp2 |
| ---- | ----- | ----- | --------- | --------- |
| A    | 10    | -     | 11/3/21   | -         |
| B    | 15    | 6     | 12/3/21   | 13/3/21   |

And of course, with the alternate table we end up with something equally convulsed

**Example 2a**

| exp | samp_A | samp_B | date_SampA | dateSampB | 
| --- | ------ | ------ | ---------- | ---------- |
| 1   | 10     | 15     | 11/3/21    | 12/3/21    |
| 2   | -      | 6      | -          | 13/3/21    |

Of course, the logical conclusion of this is that you will have a lot of named columns which are related to some other columns, but not all. This makes it much harder to programmatically wrangle.

## Data structure and semantics
We usually arrange our data as a collection of *rows* and *columns* forming a table. The examples above are transposed versions of each other, and a common way to arrange experimental data. 

The dataset is then a collection of *values* that are numeric or qualitative, which generally represent an *observation*. In our particular example we have some variables and observations which are:

1. **Experiment**: we have two experiments: 1 and 2.
2. **Sample**: We have samples A and B.
3. **Results**: various results which are related to the intersection of the sample an experiment.
4. **Date**: This one is a marker/metadata which can help browse and store a condition of the experiment. Of course, realistically you will store things like 'power', 'error rate', etc. 

Everything, including the missing value (an experiment that is yet to be made) is important to this dataset and should be retained. Of course, it may be possible to also have data that cannot ever be gathered, such as atomic distance when studying a single atom, might be unnecessary.

## Tidy data paradigm
This paradigm was first formalized by Hadley Wickham and attempts to make a easily processed structure. The idea is that one maps the meaning of a dataset to its structure, and attempts to maximise the ratio of variable columns to observational row. 
1. Each variable forms a column.
2. Each observation forms a row.
3. Each type of observational unit forms a table. 
If we apply this philosophy to the data we have we get:

**Example 3** - 'tidy data'

| experiment | Sample | date    | Result |
| ---------- | ------ | ------- | ------ |
| 1          | A      | 11/3/21 | 10     |
| 1          | B      | 12/3/21 | 15     |
| 2          | A      | -       | -      |
| 2          | B      | 13/3/21 | 6      |

With this small change we are now free from the cruel constraints of excessive columns and confusing organization. Now to select subsets of our results we can simply ask 

'*give me the rows with sample  A and experiment 1*', 

rather than **example 2**, which would require a convoluted query like:

'*give me column of sample A and the column date_sampA selecting the rows of the column of experiment  with the value 1*'. 

Of course, the complexity of the latter query would only increase, as all the columns related to the sample (or experiment, in the case of the first **example 1**) that you want would need to be explicitly called for. 

We  suffer a little bit of inefficiency, instead of 10 entries as in **1b, 2a**, we have 16 entries, but when storing such datasets, a binary tree is usually constructed during compression that represents highly duplicated values. Multi-indexing is also possible, where you have a hierarchical setup of labels, but these only become an issue when dealing with gargantuan datasets. 

Using a tidy data paradigm easily allows one to have a standard way to query and analyze data, from easily generating statistical deductions, finding trends and plotting said things. When data is not structured this way, one will need to craft bespoke strategies to extract desired values, which incurs a large overhead. This can be in either crafting specific functions to query your database or doing it manually in every case, which will make looking at your results painful, and possibly limit your exploration to what is strictly necessary or pre-planned. This invites the possibility for many errors to creep in. 

Structuring values like this comes with more advantages, many analysis operations require all the values of a variable, such as the average of all results of experiment 1 of sample A. Such operations are known as *aggregation*. Furthermore, the resulting 'row vectors' from queries of the dataset are particularly suited to vectorization, speeding up a lot of these operations. 

# Choosing appropriate orders variables and labels


#### Order
The order of the columns and rows is rather unimportant programmatically, but clearly it is sometimes useful to look at your database in person, so some care should be taken. In **3**, for example, it may be that *date* should be the last column, since it's probably not more interesting than the result. 


| experiment | Sample | date    | Result |
| ---------- | ------ | ------- | ------ |
| 1          | A      | 11/3/21 | 10     |
| 1          | B      | 12/3/21 | 15     |
| 2          | A      | -       | -      |
| 2          | B      | 13/3/21 | 6      |

Good practice dictates that *fixed values*, so called due to the fact that they are fixed by the design of the data collection, come first. An example of this from our test data is the type of experiment and sample, these are known as 'dimension' in computer science, 'subscripts on random variables' to statisticians, and 'parameters' to us. This is then followed by measured variables, and if necessary, metadata at the end. 

Metadata could be uncontrolled experimental conditions (i.e. time since the last calibration and the calibration parameters for a quantum computer - T1, T2 times per qubit, error rates, etc. Of course, this can be a link to a file containing the calibration data).  Generally you try to order rows and measurement columns such that the related variables are contiguous. This is not strictly necessary, but is definitely helpful.

## Variables and labels
Sometimes it's easy to figure out what is a variable and what is a measurement, but it's not always so obvious on how to differentiate them. For example, if you had **volume** and **mass**, these would clearly be two columns, but if you had **X position**, **Y position**, **Z position** would you have two columns or would you make a label column (**Position**) and a value column (**Position_Value**)?


| experiment | X   | Y   | Z   | result |
| ---------- | --- | --- | --- | ------ |
| 1          | 2   | 3   | 4   | 5      | 

or

| experiment | Position coordinate | position value | result |
| ---------- | ------------------- | -------------- | ------ |
| 1          | X                   | 2              | 5      |
| 1          | Y                   | 3              | 5      |
| 1          | Z                   | 4              | 5      | 

Of course, in this example, it is clear that the former is best, but cases like this might come up, and the latter arrangement may be useful if for whatever reason you only want to see what happens as you vary **X**.

In general you want to make it as easy as possible to describe functional relationships between variables between columns and comparisons between groups of columns. 

## Cleaning up data
There are several common 'problems' that are commonly found and that you might find yourself and their 'easy' solutions.

- multiple variables are stored in one column
- variables are both in rows and columns
- multiple types of unrelated observations are in the same table
- data is spread over various files

| problem                            | Solution            |
| ---------------------------------- | ------------------- |
| Column titles are values           | melting             |
| Multiple variables in one column   | string splitting    |
| variables in both rows and columns | melting + recasting |
| unrelated measurements             | filter + recreate   |
| data spread between files          | concatinate         | 

# Examples 

#### Column titles are values

**Raw data**

| row | <10 | 10-20 | >20 |
| --- | --- | ----- | --- |
| A   | 1   | 2     | 3   |
| B   | 4   | 5     | 6   |
| C   | 7   | 8     | 9   |

The general technique to change this type of data is always called melting, and functions automatically generate something like this:


**Molten data**

| row | column | value |
| --- | ------ | ----- |
| A   | <10    | 1     |
| B   | <10    | 2     |
| C   | <10    | 3     |
| A   | 10-20  | 4     |
| B   | 10-20  | 5     |
| C   | 10-20  | 6     |
| A   | >20    | 7     |
| B   | >20    | 8     |
| C   | >20    | 9     | 

Of course, in this case, the raw data form is exceptionally useful if one can perform matrix transformations on it, but it is generally better to have a method to filter out a dataset and generate the matrix-form data on call when that kind of processing is required. 

#### Multiple values in one entry

| year | column | cases |
| ---- | ------ | ----- |
| 2000 | m1524  | 0     |
| 2000 | m2534  | 1     |
| 2000 | m3544  | 0     |
| 2000 | m4554  | 0     |
| 2000 | m5564  | 0     |
| 2000 | m65    | 0     |
| 2000 | f014   | 3     |

We can split such a composite column with some clever regex or coding, but whatever the case, you will want to end up with something like this:


| year | sex | age   | cases |
| ---- | --- | ----- | ----- |
| 2000 | m   | 15–24 | 0     |
| 2000 | m   | 25–34 | 1     |
| 2000 | m   | 35–44 | 0     |
| 2000 | m   | 45–54 | 0     |
| 2000 | m   | 55–64 | 0     |
| 2000 | m   | 65+   | 0     |
| 2000 | f   | 0-14  | 3     |


#### Variables in both rows and columns

   | year | month | element | day1 | day2 | day3 | day4 | ... |
   | ---- | ----- | ------- | ---- | ---- | ---- | ---- | --- |
   | 2010 | 1     | tmax    |      |      |      |      |     |
   | 2010 | 1     | tmin    |      |      |      |      |     |
   | 2010 | 2     | tmax    | 27.3 | 24.1 |      |      |     |
   | 2010 | 2     | tmin    | 14.4 | 14.4 |      |      |     |
   | 2010 | 3     | tmax    |      |      |      | 32.1 |     |
   | 2010 | 3     | tmin    |      |      |      | 14.  |     |


A simple melt with a function to convert year+day+month into a date gives 


| date       | element | value |
| ---------- | ------- | ----- |
| 2010-01-30 | tmax    | 27.8  |
| 2010-01-30 | tmin    | 14.5  |
| 2010-02-02 | tmax    | 27.3  |
| 2010-02-02 | tmin    | 14.4  |
| 2010-02-03 | tmax    | 24.1  |
| 2010-02-03 | tmin    | 14.4  | 



but we can do a little better by **unstacking** or **casting**.

| date       | tmax | tmin |
| ---------- | ---- | ---- |
| 2010-01-30 | 27.8 | 14.5 |
| 2010-02-02 | 27.3 | 14.4 |
| 2010-02-03 | 24.1 | 14.4 |
| 2010-02-11 | 29.7 | 13.4 |
| 2010-02-23 | 29.9 | 10.7 |


#### Unrelated measurements in the same table
Here you have a complex situation, but it is fairly easy to remedy. You probably want to create or choose an ID column, which relates to the item which you have the unrelated measurements, then create two (or more) tables that have an ID column and the relevantly grouped measurements. This is important when the number of repetitions due to the unrelated observables duplicate certain entries in different amounts, which could destroy the reliability of statistical processing.


#### One type of data in multiple tables
This is very common, especially if every experiment writes to a file automatically, and you need to write and gather everything. The sequence to do this is quite simple:
1. loop over the files and create a list of tables
2. for each table, add a new column that records the filename (or some other unique identifier)
3. combine

## Easy to Implement Manipulation

Once we have this we have the ability to do several things:
- Filter and select
    - Subsets or removal of values given a condition
- Transformations
    - Can apply functions taking one or more columns and appending / replacing a column with the result



- Aggregation
    - collapsing multiple values into single values (statistical measures or any function you might want)
- Sorting 